In [ ]:
#import tensorflow as tf
import numpy as np
#import keras_cv
import cv2
#from tensorflow import keras
from matplotlib import pyplot as plt
import carla
import time
from ultralytics import YOLO
import torch, torchvision

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
spectator = world.get_spectator()
model_input_shape = 512

def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def resize_image(image):
    if max(image.shape[0], image.shape[1]) > model_input_shape:
        if image.shape[0] > image.shape[1]:
            height = model_input_shape
            width = int(image.shape[1] * model_input_shape / image.shape[0])
        else:
            height = int(image.shape[0] * model_input_shape / image.shape[1])
            width = model_input_shape
    resized_image = cv2.resize(image, (int(width), int(height)))
    resized_image = cv2.copyMakeBorder(resized_image, 0, model_input_shape - resized_image.shape[0], 0, model_input_shape - resized_image.shape[1], cv2.BORDER_CONSTANT, value=[0, 0, 0])
    scale = model_input_shape / max(image.shape[0], image.shape[1])
    return scale, resized_image

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '90')
camera_bp.set_attribute('sensor_tick', '0.0')
camera_bp.set_attribute('motion_blur_intensity', '0.0')
spawn_point = carla.Transform()

#yolov8_backbone = keras_cv.models.YOLOV8Backbone.from_preset('yolo_v8_s_backbone_coco',load_weights=True)
yolov8_plates_detector = YOLO("runs/detect/train8/weights/best.pt")

#yolov8_plates_detector = keras_cv.models.YOLOV8Detector(backbone=yolov8_backbone,num_classes=1,bounding_box_format='xywh')
#yolov8_plates_detector.compile(optimizer='adam', classification_loss='binary_crossentropy', box_loss='ciou')
model_input_shape = 512
#yolov8_plates_detector.load_weights("model_checkpoints_3.8/checkpoint_epoch_24.weights.h5")

In [10]:
ego_vehicle = spawn_vehicle(spawn_index=7)
static_vehicle = spawn_vehicle(spawn_index=9)

camera = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=1.7, z=1.2), carla.Rotation(pitch=-10)), attach_to=ego_vehicle)
video_output = np.zeros((model_input_shape, model_input_shape, 4), dtype=np.uint8)
saved_location = carla.Location(x=0, y=0, z=0)

def camera_callback(image):
    global video_output
    output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    cv2.imwrite("test.png", output)
    img = cv2.imread("test.png", cv2.IMREAD_COLOR)
    #scale, resized_image = resize_image(img)
    #cv2.imwrite("rescaled_test.png", resized_image)
    #img2 = cv2.imread("rescaled_test.png")
    #pred = yolov8_plates_detector("rescaled_test.png")
    #rescaling_factor = 1 / scale
    #for result in pred:
    #    boxes = result.boxes
    #    if boxes:
    #        cv2.rectangle(img, (int(boxes.xyxy[0][0].item() * rescaling_factor), int(boxes.xyxy[0][1].item() * rescaling_factor)), (int((boxes.xyxy[0][2].item() * rescaling_factor)), int((boxes.xyxy[0][3].item() * rescaling_factor))), (0, 255, 0), 1)
    #for box in pred["boxes"][0]:
    #    if box[0] == -1:
    #        break
    #    rescaling_factor = 1 / scale
    #    box = box * rescaling_factor
    #    cv2.rectangle(img, (int(box[0]), int(box[1])), (int((box[0] + box[2])), int((box[1] + box[3]))), (0, 255, 0), 1)
    video_output = img


camera.listen(lambda image: camera_callback(image))

ego_vehicle.set_autopilot(True)
static_vehicle.set_autopilot(True)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        if video_output is not None:
            cv2.imshow('RGB Camera', video_output)
        if cv2.waitKey(1) == ord('s'):
            saved_location = ego_vehicle.get_location()
            print("Saved location: ", saved_location)
except Exception as e:
    print(f"Error occurred: {e}")
finally:
    print("Shutting down...")
    cv2.destroyAllWindows()
    if 'camera' in locals() and camera.is_alive:
        camera.destroy()
    for actor in world.get_actors():
        if actor.type_id.startswith('vehicle.') or actor.type_id.startswith('sensor.camera'):
            actor.destroy()


Shutting down...


In [37]:
for vehicle in world.get_actors().filter('vehicle.*'):
    vehicle.destroy()


In [50]:
world.set_weather(carla.WeatherParameters.ClearNight)

In [1]:
import torch, torchvision
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torchvision.__version__)


True
11.8
90100
0.19.1+cu118
